# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [309]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
import holoviews as hv

In [310]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [311]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [312]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [313]:
to_data

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
year,,,,,,,,,,,,
2001,Agincourt North,3715,1480,0,1055,1295,195,185,5,200388,810,870
2001,Agincourt South-Malvern West,3250,1835,0,545,455,105,425,0,203047,806,892
2001,Alderwood,3175,315,0,470,50,185,370,0,259998,817,924
2001,Annex,1060,6090,5,1980,605,275,3710,165,453850,1027,1378
2001,Banbury-Don Mills,3615,4465,0,240,380,15,1360,0,371864,1007,1163
...,...,...,...,...,...,...,...,...,...,...,...,...
2016,Wychwood,920,1295,0,880,290,395,2080,35,787760,1864,1146
2016,Yonge-Eglinton,1400,1995,0,465,60,310,1445,0,1127052,2398,1535
2016,Yonge-St.Clair,520,4315,0,450,220,130,1370,0,1131888,2192,1619


In [314]:
# Calculate the sum number of dwelling types units per year (hint: use groupby)
sum_dwelling_types_year = (
    to_data[['single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house']]
    .groupby('year')
    .sum()
)

In [315]:
# Getting the data from the top 10 expensive neighbourhoods
top_10_expensive_neighbourhoods = (
    to_data.groupby('neighbourhood')
    .mean()
    .sort_values('average_house_value', ascending=False)
    .iloc[0:10 , :]
)

In [316]:
# Calculate the mean number of dwelling types units per year
mean_dwelling_types = (
    to_data[['neighbourhood', 'shelter_costs_owned', 'shelter_costs_rented']]
    .groupby('year')
    .mean()
)

In [317]:
# Calculate the average monthly shelter costs for owned and rented dwellings
monthly_avg_costs_dwellings = (
    to_data[['shelter_costs_owned', 'shelter_costs_rented']]
    .groupby('year')
    .mean()
)

In [318]:
# Calculate the average house value per year
avg_house_value = (
    to_data[['average_house_value']]
    .groupby('year')
    .mean()
)

In [319]:
# Fetch the data of all dwelling types per year
dwelling_types_year = to_data[['neighbourhood', 'single_detached_house', 'apartment_five_storeys_plus', 'movable_dwelling', 'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less', 'other_house']]

In [320]:
# Calculate the mean house values by neighbourhood per year
avg_value_neighbourhood_year = (
    to_data[['neighbourhood', 'average_house_value']]
)

### Panel Visualization Functions

In [342]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    # YOUR CODE HERE!

def create_bar_chart(year, data, color):
    """
    Create a barplot based in the data argument.
    """
    data=sum_dwelling_types_year.iloc[year].hvplot.bar(
        title=f'Dwelling Types in Toronto ({data.index[year]})',
        xlabel=f'{data.index[year]}',
        ylabel='Dwelling Type Units',
        color=color,
        rot=90,
        yformatter='%.0f'

    )
    return data

def create_line_chart(data, title, color):
    """
    Create a line chart based in the data argument.
    """
    data=data.hvplot.line(
        title=title,
        xlabel='Year',
        ylabel='Cost ($)',
        color=color,
        yformatter='%.0f'
    )
    return data

def average_house_value(data):
    """Average house values per year."""
    data=avg_house_value.hvplot.line(
        x='year',
        y='average_house_value',
        title='Average House Value Per Year',
        xlabel='House Value ($)',
        ylabel='Year',
        color='blue',
        yformatter='%.0f'
    )
    return data

def average_value_by_neighbourhood(data):
    """Average house values by neighbourhood."""
    data=avg_value_neighbourhood_year.hvplot.line(
        x='year',
        y='average_house_value',
        title='Average House Value Per Year by Neighbourhood',
        xlabel='Year',
        ylabel='Average House Value ($)',
        color='blue',
        groupby='neighbourhood',
        yformatter='%.0f'
    )
    return data

def number_dwelling_types(data):
    """Number of dwelling types per year"""
    data=dwelling_types_year.hvplot.bar(
        title='Dwelling Types Per Neighbourhood',
        xlabel='Year',
        ylabel='Dwelling Type',
        groupby='neighbourhood',
        color='blue',
        yformatter='%.0f',
        rot=90
    )
    return data
    
def average_house_value_snapshot(data):
    data=avg_house_value.hvplot.bar(
        x='year',
        y='average_house_value',
        title='Average House Value Across Toronto Per Year',
        xlabel='House Value ($)',
        ylabel='Year',
        groupby='year',
        color='blue',
        yformatter='%.0f'
    )
    return data

def top_most_expensive_neighbourhoods(data):
    data=top_10_expensive_neighbourhoods[['average_house_value']].hvplot.bar(
        rot=90,
        title='Average House Value by Neighbourhood',
        xlabel='Neighbourhood',
        yformatter='%.0f',
        ylabel='Value ($)',
        color='blue',
        height=500
    )
    return data

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [357]:
# Create a Title for the Dashboard
dash_title = '# Toronto Real Estate Analysis 2001-2016'

# Define a welcome text
welcome_text = '# Welcome to an interactive analysis of Real Estate in Toronto.'

# Create panels to structure the layout of the dashboard
# Dwelling ypes Panel
dwelling_types_panel = pn.Column(
    "## Dwelling Types in Toronto by Year",
    create_bar_chart(0,
                     sum_dwelling_types_year,
                     'blue'),
    create_bar_chart(1,
                     sum_dwelling_types_year,
                     'blue'),
    create_bar_chart(2,
                     sum_dwelling_types_year,
                     'blue'),
    create_bar_chart(3,
                     sum_dwelling_types_year,
                     'blue')
    )

# Monthly costs of Dwellings panel
monthly_costs_panel_panel = pn.Row(
    create_line_chart(
        data=monthly_avg_costs_dwellings['shelter_costs_owned'],
        title='Monthly Average Cost for Owned Dwellings in Toronto',
        color='blue'
    ),
    create_line_chart(
        data=monthly_avg_costs_dwellings['shelter_costs_rented'],
        title='Monthly Average Cost for Rented Dwellings in Toronto',
        color='red'    )
)

# Average house value panel
average_house_value_panel = pn.Column(
    average_house_value(avg_house_value)
)    
    
average_value_by_neighbourhood_panel = pn.Column(
    average_value_by_neighbourhood(avg_value_neighbourhood_year)
)

average_house_value_snapshot_panel = pn.Column(
    average_house_value_snapshot(dwelling_types_year)
)

top_most_expensive_neighbourhoods_panel = pn.Column(
    top_most_expensive_neighbourhoods(top_10_expensive_neighbourhoods[['average_house_value']])
)

In [344]:
# Create a tab layout for the dashboard
neighbourhood_pop_dashboard = pn.Tabs(
    ('Neighbourhood Analysis in Map Plots', neighbourhood_bar_1), ('Neighbourhood Analysis in Graphs', neighbourhood_2)
)

In [358]:
# Create the main dashboard
dashboard = pn.Tabs(
    ('Welcome', welcome_text),
    ('Dwelling Types', dwelling_types),
    ('Monthly Costs', monthly_costs_panel),
    ('Average House Value', average_house_value_panel),
    ('Average House Value by Neighbourhood', average_value_by_neighbourhood_panel),
    ('Average House Value in Toronto', average_house_value_snapshot_panel),
    ('10 Most Expensive Neighbourhoods', top_most_expensive_neighbourhoods_panel)
)

## Serve the Panel Dashboard

In [359]:
dashboard.servable()

Tabs
    [0] Markdown(str, name='Welcome')
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Bars)
        [3] HoloViews(Bars)
        [4] HoloViews(Bars)
    [2] Row
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
    [3] Column
        [0] HoloViews(Curve)
    [4] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                [1] VSpacer()
    [5] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] DiscreteSlider(margin=(20, 20, 20, 20), name='year', options=OrderedDict([('2001', ...]), value=2001, width=250)
                [1] VSpacer()
    [6] Column
        [0] HoloViews(Bars)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [347]:
# neighbourhood_map().show()

In [348]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [349]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [350]:
# average_house_value()

In [351]:
# average_value_by_neighbourhood()

In [352]:
# number_dwelling_types()

In [353]:
# average_house_value_snapshot()

In [354]:
# top_most_expensive_neighbourhoods()

In [355]:
# sunburts_cost_analysis()